In [ ]:
from common.helpers import load_config, load_contract, retry_call_wrapper
from live_trader.order import Order

from web3 import Web3

import datetime
from functools import lru_cache
import json
import logging
import pandas as pd
from timer import timer

In [ ]:
logging.basicConfig(level=logging.INFO)
timer.set_level(logging.INFO)

config = load_config()
# web3 = Web3(Web3.WebsocketProvider(config['rpc_endpoint']['wss']['dwellir']))
web3 = Web3(Web3.HTTPProvider(config['rpc_endpoint']['http']['dwellir']))
cfg = config['venue']['stellaswap']
router = load_contract(
    cfg['contract_address']['router'],
    cfg['abi_path']['router'],
    web3,
)
factory = load_contract(
    cfg['contract_address']['factory'],
    cfg['abi_path']['factory'],
    web3,
)
token = load_contract(
    cfg['contract_address']['token'],
    cfg['abi_path']['token'],
    web3,
)
wallet_address = '0x05a81d8564a3eA298660e34e03E5Eff9a29d7a2A'

In [ ]:
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
block_latest = 1950781 #'latest'
token_addrs = config['venue']['stellaswap']['token_trading_universe']
dot_usd_value = 6.43e-10 # changes daily but usually is around this
glmr_usd_value = 0.4617e-18 # changes daily but usually is around this
for token_addr in token_addrs:
    token = load_contract(token_addr, cfg['abi_path']['token'], web3,)
    start_balance = token.functions.balanceOf(wallet_address).call(block_identifier=1923058)
    cur_balance = token.functions.balanceOf(wallet_address).call(block_identifier=block_latest)
    if token_addr == '0xAcc15dC74880C9944775448304B263D191c6077F':
        glmr_usd_delta = (cur_balance - start_balance) * glmr_usd_value
    if token_addr == '0xFfFFfFff1FcaCBd218EDc0EbA20Fc2308C778080':
        dot_usd_delta = (cur_balance - start_balance) * dot_usd_value
    pct_delta = 100 * (cur_balance - start_balance) / start_balance
    print(f'{token_addr}: delta={pct_delta:0.2f}%, cur={cur_balance}, start={start_balance}')
    # token.functions.allowance(wallet_address, '0x70085a09D30D6f8C4ecF6eE10120d1847383BB57').call()
# Started with $40.14 of WGLMR and $52.88

start_glmr_balance = web3.eth.get_balance(wallet_address, block_identifier=1923058)
end_glmr_balance = web3.eth.get_balance(wallet_address, block_identifier=block_latest)
print('Start: GLMR balance =', start_glmr_balance, 'nonce =', web3.eth.get_transaction_count(wallet_address, 1923058))
print('End: GLMR balance =', end_glmr_balance, 'nonce =', web3.eth.get_transaction_count(wallet_address, block_latest))

print(f'Total USD delta (no fees): ${dot_usd_delta + glmr_usd_delta:0.2f} (GLMR delta = ${glmr_usd_delta:0.2f}, DOT delta = ${dot_usd_delta:0.2f})')
glmr_usd_delta += (end_glmr_balance - start_glmr_balance) * glmr_usd_value
print(f'Total USD delta (with fees): ${dot_usd_delta + glmr_usd_delta:0.2f} (GLMR delta = ${glmr_usd_delta:0.2f}, DOT delta = ${dot_usd_delta:0.2f})')

In [ ]:
print('GLMR balance =', web3.eth.get_balance(wallet_address, block_identifier=1923058) / 1e18, 'nonce =', web3.eth.get_transaction_count(wallet_address, 1923058))
print('GLMR balance =', web3.eth.get_balance(wallet_address, block_identifier='latest') / 1e18, 'nonce =', web3.eth.get_transaction_count(wallet_address))

In [ ]:
txn_hash = '0x6d5e373e7944b63f249823ce0d7bdd3f1fc608952ebc017e2a73e6e6e688c732'
web3.eth.get_transaction_receipt(txn_hash)

In [ ]:
wglmr = load_contract(
    '0xAcc15dC74880C9944775448304B263D191c6077F',
    cfg['abi_path']['token'],
    web3,
)

In [ ]:
order = Order(
    amount_in=2293619362107510,
    amount_out_min=1,
    path=[
        '0xAcc15dC74880C9944775448304B263D191c6077F',
        '0x0E358838ce72d5e61E0018a2ffaC4bEC5F4c88d2',
        # '0x818ec0A7Fe18Ff94269904fCED6AE3DaE6d6dC0b',
        # '0xfA9343C3897324496A05fC75abeD6bAC29f8A40f',
        '0xAcc15dC74880C9944775448304B263D191c6077F'
    ],
    to='0x05a81d8564a3eA298660e34e03E5Eff9a29d7a2A',
    deadline=1763107864,
    order_id=2,
    metadata={'input_usd_notional': 12.085636165564067, 'expected_out_amount': 3.955491491353395e+16, 'expected_pnl': 0.020842443088096307}
)
order = Order(
    amount_in=2293619362107510,
    amount_out_min=1,
    path=[
        '0xAcc15dC74880C9944775448304B263D191c6077F',
        '0x0E358838ce72d5e61E0018a2ffaC4bEC5F4c88d2',
        '0x818ec0A7Fe18Ff94269904fCED6AE3DaE6d6dC0b',
        '0xfA9343C3897324496A05fC75abeD6bAC29f8A40f',
        '0xAcc15dC74880C9944775448304B263D191c6077F'
    ],
    to='0x05a81d8564a3eA298660e34e03E5Eff9a29d7a2A',
    deadline=1763107864,
    order_id=2,
    metadata={'input_usd_notional': 12.085636165564067, 'expected_out_amount': 3.955491491353395e+16, 'expected_pnl': 0.020842443088096307}
)
nonce = web3.eth.get_transaction_count(wallet_address)
f = router.functions.swapExactTokensForTokens(
    order.amount_in,
    order.amount_out_min,
    order.path,
    order.to,
    order.deadline,
)

In [ ]:
txn = f.build_transaction({
    # must specify from or you'll get a ValueError:
    # {'code': -32603, 'message': 'execution fatal: Module(ModuleError [...])'}
    'from': wallet_address,
    'nonce': nonce,
    'maxFeePerGas': int(200 * 1e9),
    'maxPriorityFeePerGas': int(3 * 1e9),
})

In [ ]:
txn.update({'nonce': 21})
txn

In [ ]:
tokens = cfg['token_trading_universe']

In [ ]:
token = load_contract(
    tokens[3],
    cfg['abi_path']['token'],
    web3,
)
wallet = config['strategy']['cyclic_arbitrage']['wallet_address']
token.functions.balanceOf(wallet).call()

In [ ]:
web3.eth.account.sign_transaction

In [ ]:
%%time
receipt = web3.eth.get_transaction_receipt('0x75106fc66c0aa6539d76f63cbbe0a16d23a103d564ba46f4aa0f53f59dde330a')
receipt.logs[3]
# status = 0 -> txn failed; status = 1 -> txn succeeded
# r = web3.eth.get_transaction_receipt('0x75106fc66c0aa6539d76f63cbbe0a16d23a103d564ba46f4aa0f53f59dde330a')
# l = r.logs

In [ ]:
# Ignore the mint (addLiquidity) and burn (removeLiquidity) events
sync_filter = pair.events.Sync.createFilter(fromBlock=1625297, toBlock=1625297)
x = sync_filter.get_all_entries()[0]
x

In [ ]:
# direction doesn't matter because Stellaswap stores both (token0, token1) -> pair and (token1, token0) -> pair
factory.functions.getPair('0x0E358838ce72d5e61E0018a2ffaC4bEC5F4c88d2', '0xAcc15dC74880C9944775448304B263D191c6077F').call()

In [ ]:
factory.functions.getPair('0x0E358838ce72d5e61E0018a2ffaC4bEC5F4c88d2', '0x511aB53F793683763E5a8829738301368a2411E3').call()

In [ ]:
df = pd.read_csv("out/stellaswap_metadata_snapshot.csv")
df